# Introduction to Qiskit Functions

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.2
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.1
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.8.1
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.15.1
pyscf~=2.8.0; sys.platform != 'win32'
```
</details>

<Admonition type="note">
Qiskit Functions are an experimental feature available only to IBM Quantum&trade; Premium Plan users. They are in preview release status and subject to change.
</Admonition>

Qiskit Functions simplify and accelerate utility-scale algorithm discovery and application development, by abstracting away parts of the quantum software development workflow. In this way, Qiskit Functions free up time normally spent hand-writing code and fine-tuning experiments.

![Overview of Qiskit Functions](/images/guides/functions/functions-overview.svg)
Functions come in two forms:

| Type | What does it do? | Example inputs and outputs | Who is it for? |
| - | - | - | - |
| Circuit function | Simplified interface for running circuits. Abstracts transpilation, error suppression, error mitigation | **Input**: Abstract `PUB`s <br/> **Output**: Mitigated expectation values | Researchers using Qiskit to discover new algorithms and applications, without needing to focus on optimizing for hardware or handling error. Circuit functions can be used to build custom application functions. |
| Application function | Covers higher-level tasks, like exploring algorithms and domain-specific use cases. Abstracts quantum workflow to solve tasks, with classical inputs and outputs | **Input**: Molecules, graphs <br/> **Output**: Energy, cost | Researchers in non-quantum domains, integrating quantum into existing large-scale classical workflows, without needing to map classical data to quantum circuits. |

Functions are provided by IBM&reg; and third-party partners. Each is performant for specific workload characteristics and have unique performance-tuning options. Premium Plan users can get started with IBM Qiskit Functions for free, or procure a license from one of the partners who have contributed a function to the catalog.

## Get started with Qiskit Functions

### Install Qiskit Functions Catalog client

1. To start using Qiskit Functions, install the IBM Qiskit Functions Catalog client:

    ```
    pip install qiskit-ibm-catalog
    ```
1. Retrieve your API token from the [IBM Quantum account page](https://quantum.ibm.com/account), and activate your Python virtual environment.  See the [installation instructions](/guides/install-qiskit#local) if you do not already have a virtual environment set up.

    <span id="save-account"></span>**If you are working in a trusted Python environment (such as on a personal laptop or workstation),** use the `save_account()` method to save your credentials locally. ([Skip to the next step](#functions-untrusted) if you are not using a trusted environment, such as a shared or public computer, to authenticate to IBM Quantum Platform.)

    To use `save_account()`, run `python` in your shell to open a REPL (read-eval-print loop), then enter the following:

    ```python
    from qiskit_ibm_catalog import QiskitFunctionsCatalog

    QiskitFunctionsCatalog.save_account(token="<your-token>")
    ```

    Close out of the REPL with `exit()`. From now on, whenever you need to authenticate to the service, you can load your credentials with `QiskitFunctionsCatalog()`.

In [1]:
# Load saved credentials
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()

3. <span id="functions-untrusted"></span>**Avoid executing code on an untrusted machine or an external cloud Python environment to minimize security risks.** If you must use an untrusted environment (on, for example, a public computer), change your API token after each use by expiring it on the [IBM Quantum Platform dashboard](https://quantum.ibm.com/) (click the refresh button in the API token field) to reduce risk. To initialize the service in this situation, expand the following section to view code you can use:

    <details>
        <summary>Initialize the service in an untrusted environment</summary>

    ```python
    from qiskit_ibm_catalog import QiskitFunctionsCatalog

    # After using the following code, go to your dashboard (https://quantum.ibm.com/)
    # and expire your API token (click the refresh button in the API token field)
    catalog = QiskitFunctionsCatalog(token="<MY_IBM_QUANTUM_TOKEN>")
    ```
    </details>

    <Admonition type="caution">
      **Protect your API token!** Never include your token in source code, Python script, or notebook file. When sharing code with others, ensure that your API token is not embedded directly within the Python script. Instead, share the script without the token and provide instructions for securely setting it up.

      If you accidentally share your token with someone or include it in version control like Git, immediately revoke your token by expiring it on the [IBM Quantum Platform dashboard](https://quantum.ibm.com/) (click the refresh button in the API token field) to reduce risk.
    </Admonition>

4. Once you have authenticated, you can list the functions from the Qiskit Functions Catalog that you have access to:

In [2]:
catalog.list()

[QiskitFunction(qedma/qesem-dev),
 QiskitFunction(qunasys/quri-chemistry),
 QiskitFunction(algorithmiq/tem),
 QiskitFunction(algorithmiq/tem-gpu),
 QiskitFunction(qedma/qesem),
 QiskitFunction(multiverse/singularity),
 QiskitFunction(q-ctrl/performance-management-dev),
 QiskitFunction(qunasys/quri-chemistry-dev),
 QiskitFunction(ibm/circuit-function),
 QiskitFunction(q-ctrl/optimization-solver),
 QiskitFunction(q-ctrl/performance-management),
 QiskitFunction(algorithmiq/tem-dev),
 QiskitFunction(q-ctrl/optimization-solver-dev)]

### Run enabled functions

Once a catalog object has been instantiated, you can select a function using `catalog.load(provider/function-name)`:

In [3]:
ibm_cf = catalog.load("ibm/circuit-function")

Each Qiskit Function has custom inputs, options, and outputs. Check the specific documentation pages for the function you want to run for more information. By default, all users can only run one function job at a time:

In [4]:
# This cell is hidden from users
# It gets these details programatically so we can test this notebook
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.random import random_circuit

service = QiskitRuntimeService()
instance = service.active_account()["instance"]
backend_name = service.least_busy().name

circuit = random_circuit(num_qubits=2, depth=2, seed=42)
observable = "Z" * circuit.num_qubits

In [5]:
job = ibm_cf.run(
    pubs=[(circuit, observable)],
    instance=instance,  # E.g. "ibm-q/open/main"
    backend_name=backend_name,  # E.g. "ibm_kyiv"
)

job.job_id

'3abb3dda-6d4b-4477-a76c-c9d231f62698'

### Check job status

<Admonition type="tip">
Currently, the IBM Quantum workloads table only reflects Qiskit Runtime workloads. Use `job.status()` to see your Qiskit Function workload's current status.
</Admonition>

With your Qiskit Function `job_id`, you can check the status of running jobs. This includes the following statuses:

- **`QUEUED`**: The remote program is in the Qiskit Function queue. The queue priority is based on how much you've used Qiskit Functions.
- **`INITIALIZING`**: The remote program is starting; this includes setting up the remote environment and installing dependencies.
- **`RUNNING`**: The program is running.
- **`DONE`**: The program is complete, and you can retrieve result data with `job.results()`.
- **`ERROR`**: The program stopped running because of a problem. Use `job.result()` to get the error message.
- **`CANCELED`**: The program was canceled; either by a user, the service, or the server.

In [6]:
job.status()

'QUEUED'

### Retrieve results

Once a program is `DONE`, you can use `job.results()` to fetch the result. This output format varies with each function, so be sure to follow the specific documentation:

In [7]:
result = job.result()
print(result)

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': True, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})


At any time, you can also cancel a job:

In [8]:
job.stop()

'Job has been stopped.'

### List previously run jobs run with Qiskit Functions

You can use `jobs()` to list all jobs submitted to Qiskit Functions:

In [9]:
old_jobs = catalog.jobs()
old_jobs

[<Job | eacba5e8-3564-4738-9336-f160fa45c313>,
 <Job | b1486233-3f59-4c19-b52d-6ad729b891cc>,
 <Job | a4fb9fca-b558-4dbd-b4ce-20ebd9e91091>,
 <Job | 7b74103e-3b6a-43b6-983b-b601a2b99149>,
 <Job | 3abb3dda-6d4b-4477-a76c-c9d231f62698>,
 <Job | 3bcf1f7b-fc67-4675-a587-af699764c2b0>,
 <Job | 60f99531-a023-452d-bfa5-9b14f5fd575b>,
 <Job | 9e43a37a-b657-4637-b903-218d78581b3c>,
 <Job | 74b604b2-ea0b-4e34-b3a2-555762ba53da>,
 <Job | c58331bf-fa7c-435e-9fdf-1108bf4ce24f>]

### Fetch error messages

If a program status is `ERROR`, use `job.result()` to fetch the error message to help debug as follows:

In [10]:
print(job.result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': True, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})


## Next steps

<Admonition type="info" title="Recommendations">

- [Explore circuit functions](./ibm-circuit-function) to build new algorithms and applications, without needing to manage transpilation or error handling.
- [Explore application functions](./q-ctrl-optimization-solver) to solve domain-specific tasks, with classical inputs and outputs.

</Admonition>